In [1]:
from collections import defaultdict, Counter, OrderedDict
import ujson
import pathlib
from pathlib import Path
import sqlite3
import sys
import textwrap
import torch
from tqdm import tqdm
from typing import Dict, List, Set, Union

import unicodedata

import uuid

import numpy as np

%load_ext autoreload
%autoreload 2

from aic_nlp_utils.batch import batch_apply
from aic_nlp_utils.encoding import nfc
from aic_nlp_utils.json import read_jsonl, read_json, write_json, write_jsonl
from aic_nlp_utils.fever import fever_detokenize, import_fever_corpus_from_sqlite

# from zshot_fact_verify.qa2d.qa2d import SameDocumentNERReplacementGenerator
from zshot_fact_verify.wiki.load import load_corpus, create_corpus_splits, select_nei_context_for_splits, load_nei_ners
# from zshot_fact_verify.models.load import load_tokenizer_and_model
# from zshot_fact_verify.models.arguments import ModelArguments

In [2]:
SEED = 1234
NER_ROOT = '/mnt/data/factcheck/wiki/cs/20230220/qacg/ner/PAV-ner-CNEC'
WIKI_CORPUS = '/mnt/data/factcheck/wiki/cs/20230220/paragraphs/cswiki-20230220-paragraphs.jsonl'
SPLITS  = [
            {"name": "train", "file": Path(NER_ROOT, "train_ners.json"), "size": 10000},
            {"name": "dev", "file": Path(NER_ROOT, "dev_ners.json"), "size": 1000},
            {"name": "test", "file": Path(NER_ROOT, "test_ners.json"), "size": 1000},
        ]
corpus, corpus_id2idx, corpus_pages = load_corpus(WIKI_CORPUS)
corpus_recs_lst = create_corpus_splits(corpus, corpus_id2idx, SPLITS, SEED)
corpus_recs_lst = select_nei_context_for_splits(corpus, corpus_id2idx, corpus_recs_lst, SEED)
corpus_recs_dict = {}
for i, split in enumerate(["train", "dev", "test"]):
    crl = corpus_recs_lst[i]
    cd = {s["id"]: {"text": s["text"], "nei_text": s.get("nei_text")} for s in crl}
    corpus_recs_dict[split] = cd 

imported 514568 corpus pages.


In [3]:
corpus_recs_dict["train"]['Martin_Havelka_0']

{'text': 'Martin Havelka',
 'nei_text': 'Osobní život.\nByl ženatý. Se svou manželkou Ivou Havelkovou měl tři děti: Jana, Martina a Emílii Emmu. Mezi jeho zájmy patřilo vyřezávání dřevěných soch, vše okolo indiánů, fotografování a také ho bavil westernový život. Žil nedaleko Brna ve vesnici jménem Radostice. Manželka pracuje také v Městském divadle Brno jako rekvizitářka a jeho syn Jan pracuje tamtéž jako osvětlovač.'}

In [4]:
DATA_ROOT = '/mnt/data/factcheck/wiki/cs/20230220/qacg'
NER_DIR = 'PAV-ner-CNEC'
QG_DIR = 'mt5-large-cp59k'
QA2D_DIR = 'mbart-large-cc25_cp26k'

CLAIM_ROOT = Path(DATA_ROOT, 'claim', NER_DIR, QG_DIR, QA2D_DIR)
CLAIM_QUALITY_ROOT = Path(DATA_ROOT, 'claim_quality_v3', NER_DIR, QG_DIR, QA2D_DIR)

In [6]:
def sample_random_claims(claim_root, corpus_recs_dict, split, claim_types, n_per_split, seed=1234):
    # samples claims to be annotated in Doccano locally
    # run: doccano task
    # and: doccano webserver --port 8000 in /Users/drchajan/Downloads/doccano
    corpus_recs = corpus_recs_dict[split]
    rng = np.random.RandomState(seed)
    sel = []
    for ct in claim_types:
        claim_dict = read_json(Path(claim_root, f"{split}_{ct}.json"))
        claim_set = set()
        claims = []
        for pid_, page in claim_dict.items():
            for id_, claim in page.items():
                if claim not in claim_set:
                    claim_set.add(claim)
                    claims.append((pid_, claim))

        claims = [claims[i] for i in rng.choice(len(claims), n_per_split, replace=False)]
        for id_, c in claims:
            meta = {"claim_type": ct, "text": corpus_recs[id_]["text"]}
            if ct == "nei":
                meta["nei_text"] = corpus_recs[id_]["nei_text"]
            sel.append({"text": c, "meta": meta})
    rng.shuffle(sel)
    return sel


# claims = sample_random_claims(CLAIM_ROOT, corpus_recs_dict, "train", ["support", "refute", "nei"], n_per_split=1000)
# write_jsonl("/home/drchajan/claim_quality-mbart-large-cc25_cp26k.jsonl", claims)

In [22]:
from sklearn.model_selection import train_test_split

def create_classification_splits_from_annotated(src_jsonl, dst_dir, n_dev, n_test, all_classes=False):
    src = read_jsonl(src_jsonl)
    samples = []
    label_cnt = Counter()
    for s in src:
        if len(s['label']) > 0:
            # if len(s['label']) > 1:
                # print(s)
            assert len(s['label']) == 1, s
            label = s["label"][0]
            if not all_classes:
                if label != "ok": # move all negative samples to single "bad" class
                    label = "bad"
            label_cnt[label] += 1
            samples.append({"text": s["text"], "label": label})
    print(f"imported {len(samples)}/{len(src)} records")
    print(f"labels: {label_cnt}")
    # the samples are already shuffled for annotation - see above, but I want stratified split, so we need to shuffle once more
    dev_samples, train_samples = train_test_split(samples, train_size=n_dev, shuffle=True, stratify=[s["label"] for s in samples], random_state=1234)
    test_samples, train_samples = train_test_split(train_samples, train_size=n_test, shuffle=True, stratify=[s["label"] for s in train_samples], random_state=1234)
    print(f"train split size: {len(train_samples)}, labels {Counter([s['label']for s in train_samples])}")
    print(f"dev split size: {len(dev_samples)}, labels {Counter([s['label']for s in dev_samples])}")
    print(f"test split size: {len(test_samples)}, labels {Counter([s['label']for s in test_samples])}")
    suffix = "_all_classes" if all_classes else ""
    write_jsonl(Path(dst_dir, f"train{suffix}.jsonl"), train_samples, mkdir=True)
    write_jsonl(Path(dst_dir, f"dev{suffix}.jsonl"), dev_samples, mkdir=True)
    write_jsonl(Path(dst_dir, f"test{suffix}.jsonl"), test_samples, mkdir=True)
    print(f"saved to {dst_dir}")


create_classification_splits_from_annotated("/home/drchajan/all.jsonl", CLAIM_QUALITY_ROOT, n_dev=100, n_test=100)
create_classification_splits_from_annotated("/home/drchajan/all.jsonl", CLAIM_QUALITY_ROOT, n_dev=100, n_test=100, all_classes=True)

imported 1401/3000 records
labels: Counter({'ok': 848, 'bad': 553})
train split size: 1201, labels Counter({'ok': 727, 'bad': 474})
dev split size: 100, labels Counter({'ok': 61, 'bad': 39})
test split size: 100, labels Counter({'ok': 60, 'bad': 40})
saved to /mnt/data/factcheck/wiki/cs/20230220/qacg/claim_quality_v3/PAV-ner-CNEC/mt5-large-cp59k/mbart-large-cc25_cp26k
imported 1401/3000 records
labels: Counter({'ok': 848, 'bad_grammar': 202, 'bad': 198, 'incomplete': 153})
train split size: 1201, labels Counter({'ok': 726, 'bad_grammar': 174, 'bad': 170, 'incomplete': 131})
dev split size: 100, labels Counter({'ok': 61, 'bad_grammar': 14, 'bad': 14, 'incomplete': 11})
test split size: 100, labels Counter({'ok': 61, 'bad': 14, 'bad_grammar': 14, 'incomplete': 11})
saved to /mnt/data/factcheck/wiki/cs/20230220/qacg/claim_quality_v3/PAV-ner-CNEC/mt5-large-cp59k/mbart-large-cc25_cp26k


In [19]:
def balance_split(split_dir, src_split, dst_split, seed=1234):
    # balances by subsampling to the least present class
    recs = read_jsonl(Path(split_dir, src_split))
    label_cnts = Counter(r["label"] for r in recs)
    print(f"initial counts: {label_cnts}")
    least_occurences = label_cnts.most_common()[-1][1]
    ret = []
    for r in recs:
        l = r["label"]
        if label_cnts[l] == least_occurences:
            ret.append(r)
        else:
            label_cnts[l] -= 1
    print(f"balanced counts: {label_cnts}")
    rng = np.random.RandomState(seed)
    rng.shuffle(ret)
    dst_path = Path(split_dir, dst_split)
    write_jsonl(dst_path, ret)
    print(f"saved to: {dst_path}")

balance_split(CLAIM_QUALITY_ROOT, "train.jsonl", "train_balanced.jsonl")

initial counts: Counter({'ok': 727, 'bad': 474})
balanced counts: Counter({'ok': 474, 'bad': 474})
saved to: /mnt/data/factcheck/wiki/cs/20230220/qacg/claim_quality_v3/PAV-ner-CNEC/mt5-large-cp59k/mbart-large-cc25_cp26k/train_balanced.jsonl


In [44]:
def construct_prompt(split_dir, src_split, examples_per_cls=2, seed=1234):
    recs = read_jsonl(Path(split_dir, src_split))
    rng = np.random.RandomState(seed)
    rng.shuffle(recs)
    labels = set(r["label"] for r in recs)
    label_cnts = Counter()
    for l in labels:
        label_cnts[l] = examples_per_cls
    sel = []
    rest = []
    for r in recs:
        l = r["label"]
        if label_cnts[l] > 0:
            sel.append(r)
            label_cnts[l] -= 1
        else:
            rest.append(r)

    prompt = "Vyhodnoť následující řadu tvrzení. Cílem je rozpoznat, která z nich by bylo možné a zajímavé ověřit. Udej klasifikaci: OK/ŠPATNÉ a pro třídu ŠPATNÉ i vysvětlení:"
    for r in sel:
        l = r['label']
        prompt += f"\nTvrzení: {r['text']}"
        prompt += "\nTřída: " + ("OK" if l == "ok" else "ŠPATNÉ")
        if l == 'bad_grammar':
            prompt += "\nVysvětlení: špatná gramatika"
        elif l == 'incomplete':
            prompt += "\nVysvětlení: příliš obecné nebo neúplné"
        elif l == 'bad':
            prompt += "\nVysvětlení: celkově špatné"
        prompt += "\n"
    prompt += f"\nTvrzení: {rest[3]['text']}"
    return prompt

prompt = construct_prompt(CLAIM_QUALITY_ROOT, "train_all_classes.jsonl", examples_per_cls=3)
print(prompt)
    

Vyhodnoť následující řadu tvrzení. Cílem je rozpoznat, která z nich by bylo možné a zajímavé ověřit. Udej klasifikaci: OK/ŠPATNÉ a pro třídu ŠPATNÉ i vysvětlení:
Tvrzení: Doktor Proktor a velká loupež zlata vyšla v roce 2012.
Třída: OK

Tvrzení: František Sokol se narodil v roce 1968.
Třída: OK

Tvrzení: Světec je na obraze svatého Šebestiánu.
Třída: ŠPATNÉ
Vysvětlení: celkově špatné

Tvrzení: Vláda Ludvíka XIV. skončila v roce 74.
Třída: OK

Tvrzení: Karlo-Ferdinandova univerzita se nachází v Praze.
Třída: ŠPATNÉ
Vysvětlení: špatná gramatika

Tvrzení: Rodina bydlela na Židovském Městě pražském.
Třída: ŠPATNÉ
Vysvětlení: příliš obecné nebo neúplné

Tvrzení: Daniel Pearl se stal členem Wall Street Journal v roce 1990.
Třída: ŠPATNÉ
Vysvětlení: celkově špatné

Tvrzení: Trubadúr porazil koňskou oblast.
Třída: ŠPATNÉ
Vysvětlení: celkově špatné

Tvrzení: Bývalý pilot F1, který podpořil projekt, byl Adrian Newey.
Třída: ŠPATNÉ
Vysvětlení: příliš obecné nebo neúplné

Tvrzení: Aryna Sabalenkov